In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
from fastai.callbacks.hooks import num_features_model
from torch.nn import L1Loss
import pandas as pd
from torch import optim
import re
import json
import cv2
import types



In [ ]:
import numpy as np
import torch
import pandas as pd
import random
import string

# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def map5kfast(preds, targs, k=10):
    predicted_idxs = preds.sort(descending=True)[1]
    top_5 = predicted_idxs[:, :5]
    scores = torch.zeros(len(preds), k).float()
    for kk in range(k):
        scores[:,kk] = (top_5[:,kk] == targs).float() / float((kk+1))
    return scores.max(dim=1)[0].mean()

def map5(preds,targs):
    if type(preds) is list:
        return torch.cat([map5fast(p, targs, 5).view(1) for p in preds ]).mean()
    return map5kfast(preds,targs, 5)

def top_5_preds(preds): return np.argsort(preds.numpy())[:, ::-1][:, :5]

def top_5_pred_labels(preds, classes):
    top_5 = top_5_preds(preds)
    labels = []
    for i in range(top_5.shape[0]):
        labels.append(' '.join([classes[idx] for idx in top_5[i]]))
    return labels

def create_submission(preds, data, name, classes=None):
    if not classes: classes = data.classes
    sub = pd.DataFrame({'Image': [path.name for path in data.test_ds.x.items]})
    sub['Id'] = top_5_pred_labels(preds, classes)
    sub.to_csv(f'subs/{name}.csv.gz', index=False, compression='gzip')


def intersection(preds, targs):
    # preds and targs are of shape (bs, 4), pascal_voc format
    max_xy = torch.min(preds[:, 2:], targs[:, 2:])
    min_xy = torch.max(preds[:, :2], targs[:, :2])
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, 0] * inter[:, 1]

def area(boxes):
    return ((boxes[:, 2]-boxes[:, 0]) * (boxes[:, 3]-boxes[:, 1]))

def union(preds, targs):
    return area(preds) + area(targs) - intersection(preds, targs)

def IoU(preds, targs):
    return intersection(preds, targs) / union(preds, targs)

def name(n=10, print_it=True):
    name = "".join(random.choice(string.ascii_lowercase) for _ in range(n))
    if print_it: print(name)
    return name

In [ ]:

import fastai
from fastprogress import force_console_behavior
import fastprogress
fastprogress.fastprogress.NO_BAR = True
master_bar, progress_bar = force_console_behavior()
fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar

In [ ]:
j = json.load(open('train-object-detect\\annotations.json'))


In [ ]:
j[0]['annotations'][0]


In [ ]:

SZ = 360
BS = 12
NUM_WORKERS = 0

In [ ]:
def anno2bbox(anno):
    im_width, im_height = PIL.Image.open(f"train-object-detect\\{anno['filename']}").size
    anno = list(filter(lambda a: a['class'] == 'rect', anno['annotations']))[0]
    return [
        np.clip(anno['x'], 0, im_height) / im_height * SZ,
        np.clip(anno['y'], 0, im_width) / im_width * SZ,
        np.clip(anno['x']+anno['height'], 0, im_height) / im_height * SZ,
        np.clip(anno['y']+anno['width'], 0, im_width) / im_width * SZ
    ]

In [ ]:
fn2bbox = {jj['filename']: [[anno2bbox(jj)], ['rect']] for jj in j}
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)
get_y_func = lambda o: fn2bbox[path2fn(o)]

In [ ]:
val_fns = pd.read_pickle('val_fns_detection.pkl') # I create this file in rough object detection.ipynb


In [ ]:
class StubbedObjectCategoryList(ObjectCategoryList):
    def analyze_pred(self, pred): return [pred.unsqueeze(0), torch.ones(1).long()]

In [ ]:
data = (ObjectItemList.from_df(pd.DataFrame(data=list(fn2bbox.keys())), path='train-object-detect')
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)                         
        .label_from_func(get_y_func, label_cls=StubbedObjectCategoryList)
        .transform(get_transforms(max_zoom=1, max_warp=0.05, max_rotate=0.05, max_lighting=0.2), tfm_y=True, size=(SZ,SZ), resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS)
        .normalize(imagenet_stats))

In [ ]:
data.valid_ds.items


In [ ]:
batch = next(iter(data.valid_dl))


In [ ]:
batch[0][0][1]


In [ ]:
data.show_batch(rows=3, ds_type=DatasetType.Train, figsize=(12,12))


In [ ]:
class SnakeDetector(nn.Module):
    def __init__(self, arch=models.resnet18):
        super().__init__() 
        self.cnn = create_body(arch)
        self.head = create_head(num_features_model(self.cnn) * 2, 4)
        
    def forward(self, im):
        x = self.cnn(im)
        x = self.head(x)
        return 2 * (x.sigmoid_() - 0.5)

In [ ]:
def loss_fn(preds, targs, class_idxs):
    return L1Loss()(preds, targs.squeeze())

In [ ]:
learn = Learner(data, SnakeDetector(arch=models.resnet50), loss_func=loss_fn)
learn.metrics = [lambda preds, targs, _: IoU(preds, targs.squeeze()).mean()]

In [ ]:
learn.split([learn.model.cnn[:6], learn.model.cnn[6:], learn.model.head])


In [ ]:
learn.freeze_to(-1)


In [ ]:
%time learn.fit_one_cycle(25, 1e-2, div_factor=12, pct_start=0.2)


In [ ]:
learn.recorder.plot_losses()


In [ ]:
learn.recorder.plot_lr()


In [ ]:
learn.unfreeze()


In [ ]:
max_lr = 1e-3
lrs = np.array([max_lr/100, max_lr/10, max_lr])

In [ ]:
%time learn.fit_one_cycle(120, lrs)

In [ ]:
learn.save('snake_detector')


In [ ]:
learn.recorder.plot_losses()


In [ ]:
learn.show_results(rows=6)
